In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [9]:
# Setting the constants
BATCHSIZE, CHANNELS, WIDTH, HEIGHT = 100, 1, 28, 28
FAKE, REAL = 0, 1

In [3]:
### Downloading the MNIST dataset

!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz
transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                             ])
mnist_set = MNIST(root='./', download=True, transform=transform)
dataset = DataLoader(mnist_set, batch_size=BATCHSIZE)

--2021-04-30 11:08:04--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-04-30 11:08:04--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz            [      <=>           ]  33.20M  31.1MB/s    in 1.1s    

2021-04-30 11:08:05 (31.1 MB/s) - ‘MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/tra

In [134]:
class Discriminator(nn.Module):
  """Architecture of the discriminator that distinguishes real and generated
     images from each other."""
  def __init__(self):
    super(Discriminator, self).__init__()
    self.conv1 = nn.Conv2d(CHANNELS, 32, kernel_size=7, stride=1)
    self.batchnorm1 = nn.BatchNorm2d(32)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=8, stride=1)
    self.batchnorm2 = nn.BatchNorm2d(64)
    self.conv3 = nn.Conv2d(64, 1, kernel_size=8, stride=8)
  
  def forward(self, input):
    output = self.conv1(input)
    output = F.leaky_relu(output)
    output = self.batchnorm1(output)
    output = self.conv2(output)
    output = F.leaky_relu(output)
    output = self.batchnorm2(output)
    output = self.conv3(output)
    output = torch.sigmoid(output)
    return output

In [136]:
class Generator(nn.Module):
  """Architecture for the generator. It generates an image based on noise
     created from a zero-mean gaussian."""
  def __init__(self):
    super(Generator, self).__init__()
    self.conv1 = nn.ConvTranspose2d(BATCHSIZE, 512, 4, 1, 0)
    self.batchnorm1 = nn.BatchNorm2d(512)
    self.conv2 = nn.ConvTranspose2d(512, 256, 4, 2, 1)
    self.batchnorm2 = nn.BatchNorm2d(256)
    self.conv3 = nn.ConvTranspose2d(256, 128, 4, 2, 2)
    self.batchnorm3 = nn.BatchNorm2d(128)
    self.conv4 = nn.ConvTranspose2d(128, 64, 4, 1, 1)
    self.batchnorm4 = nn.BatchNorm2d(64)
    self.conv5 = nn.ConvTranspose2d(64, 1, 4, 2, 2)

  def forward(self, input):
    output = self.batchnorm1(F.relu(self.conv1(input)))
    output = self.batchnorm2(F.relu(self.conv2(output)))
    output = self.batchnorm3(F.relu(self.conv3(output)))
    output = self.batchnorm4(F.relu(self.conv4(output)))
    output = torch.tanh(self.conv5(output))
    return output

In [137]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
D = Discriminator().to(device)
G = Generator().to(device)
# loss function, binary cross-entropy since we have 0 and 1 as labels
bce = nn.BCELoss()
optimizerD = torch.optim.Adam(D.parameters(), lr=0.0002)
optimizerG = torch.optim.Adam(G.parameters(), lr=0.0002)
epochs = 7

for epoch in range(epochs):
  for i, data in enumerate(dataset):
    D.zero_grad()
    # train the discriminator on real numbers 
    # create labels for real images
    label = torch.full((BATCHSIZE,), REAL, dtype=torch.float, device=device)
    # make predictions with the discriminator
    predictions = D(data[0].cuda()).to(device).view(-1)
    # get and update the gradients with the real labels
    realError = bce(predictions, label)
    realError.backward()

    # train the discriminator on fake generated images
    # create the fake labels 
    label.fill_(FAKE)
    # create noise
    noise = torch.randn(BATCHSIZE, BATCHSIZE, 1, 1, device=device)
    # create a fake image from the noise
    fake_images = G(noise).to(device)
    # make predictions with the discriminator
    predictions = D(fake_images.detach()).to(device).view(-1)
    # get and update the gradients with the fake labels
    fakeError = bce(predictions, label)
    fakeError.backward()
    addedError = realError + fakeError
    optimizerD.step()

    # train the generator 
    G.zero_grad()
    # initialize real labels again
    label.fill_(REAL)
    # make predictions with the discriminator for the fake images
    predictions = D(fake_images).to(device).view(-1)
    # train the generator with these predictions and the real labels
    generatorError = bce(predictions, label)
    generatorError.backward()
    optimizerG.step()
    print('Samples: {}/60000\nEpoch: {}\nLoss D: {}\nLoss G: {}\n'.format((i+1)*BATCHSIZE, epoch, addedError, generatorError))

Streaming output truncated to the last 5000 lines.
Samples: 20100/60000
Epoch: 5
Loss D: 0.5564233660697937
Loss G: 2.568850040435791

Samples: 20200/60000
Epoch: 5
Loss D: 0.6426974534988403
Loss G: 2.583766460418701

Samples: 20300/60000
Epoch: 5
Loss D: 0.6934854984283447
Loss G: 2.385523557662964

Samples: 20400/60000
Epoch: 5
Loss D: 0.4812513291835785
Loss G: 2.6623566150665283

Samples: 20500/60000
Epoch: 5
Loss D: 0.5878605842590332
Loss G: 2.2090606689453125

Samples: 20600/60000
Epoch: 5
Loss D: 0.6595326066017151
Loss G: 2.1740901470184326

Samples: 20700/60000
Epoch: 5
Loss D: 0.39506274461746216
Loss G: 2.123553514480591

Samples: 20800/60000
Epoch: 5
Loss D: 0.3898337483406067
Loss G: 2.151378631591797

Samples: 20900/60000
Epoch: 5
Loss D: 0.5685178637504578
Loss G: 2.1316280364990234

Samples: 21000/60000
Epoch: 5
Loss D: 0.585817813873291
Loss G: 2.224043846130371

Samples: 21100/60000
Epoch: 5
Loss D: 0.5002283453941345
Loss G: 2.1927435398101807

Samples: 21200/60000

In [147]:
# plot images
noise = torch.randn(BATCHSIZE, BATCHSIZE, 1, 1, device=device)
images = G(noise).to(device)
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(images[64].detach().cpu()[0], cmap='gray')
ax2.imshow(images[32].detach().cpu()[0], cmap='gray')